In [5]:
import tensorflow as tf
import os
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, Activation
from tensorflow.keras.optimizers import Adam
from sklearn.metrics import classification_report
from tensorflow.keras.utils import Sequence

In [7]:
# 1. 224px 데이터 로더 정의 (2만장 샘플링 기능 포함)
class RandomSubsetGenerator(Sequence):
    def __init__(self, x_folder, y_path, batch_size=32, subset_size=20000):
        self.x_folder = x_folder
        self.y_full = np.load(y_path)
        self.batch_size = batch_size
        
        # 전체 데이터 중 2만 개만 무작위 선택
        total_indices = np.arange(len(self.y_full))
        np.random.seed(42) # 결과 재현을 위해 시드 고정
        self.indices = np.random.choice(total_indices, size=min(subset_size, len(self.y_full)), replace=False)
        self.total = len(self.indices)
        
        print(f"학습 데이터: 전체 {len(self.y_full)}장 중 {self.total}장 무작위 샘플링 완료")

    def __len__(self):
        return int(np.ceil(self.total / self.batch_size))

    def __getitem__(self, index):
        batch_ids = self.indices[index * self.batch_size:(index + 1) * self.batch_size]
        X = [np.load(os.path.join(self.x_folder, f"img_{i:05d}.npy")) for i in batch_ids]
        y = self.y_full[batch_ids]
        return np.array(X, dtype=np.float32), y

In [8]:
# 검증용 로더 (전체 검증 데이터 사용)
class ValGenerator(Sequence):
    def __init__(self, x_folder, y_path, batch_size=32):
        self.x_folder = x_folder
        self.y = np.load(y_path)
        self.batch_size = batch_size
        self.indices = np.arange(len(self.y))

    def __len__(self):
        return int(np.ceil(len(self.y) / self.batch_size))

    def __getitem__(self, index):
        batch_ids = self.indices[index * self.batch_size:(index + 1) * self.batch_size]
        X = [np.load(os.path.join(self.x_folder, f"img_{i:05d}.npy")) for i in batch_ids]
        y = self.y[batch_ids]
        return np.array(X, dtype=np.float32), y

In [9]:
# 2. 데이터 준비
# (폴더 경로는 사용자님 환경에 맞게 수정될 수 있습니다. Cell 34, Cell 5 참고함)
TRAIN_DIR = "full_npy_images"
TRAIN_LABEL = "y_full.npy"
VAL_DIR = r"D:\workspace\full_val_npy" # 전체 검증 데이터 폴더
VAL_LABEL = os.path.join(VAL_DIR, "y_full.npy")

train_gen_20k = RandomSubsetGenerator(TRAIN_DIR, TRAIN_LABEL, subset_size=20000)
val_gen_full = ValGenerator(VAL_DIR, VAL_LABEL)

학습 데이터: 전체 328210장 중 20000장 무작위 샘플링 완료


In [13]:
def create_original_model():
    # Cell 14의 하이퍼파라미터
    input_size = 224  # (원본 256 -> 224로 변경)
    kernel_size = 3
    filters = 8
    num_conv_layers = 3
    activation = 'relu'
    
    model = Sequential()
    
    # Conv 레이어 쌓기 (8 -> 16 -> 32)
    for i in range(num_conv_layers):
        filter_count = filters * (2 ** i)
        if i == 0:
            model.add(Conv2D(filter_count, (kernel_size, kernel_size), 
                           input_shape=(input_size, input_size, 3)))
        else:
            model.add(Conv2D(filter_count, (kernel_size, kernel_size)))
            
        model.add(Activation(activation))
        model.add(MaxPooling2D())

    model.add(Flatten())
    
    # Dense 레이어 (128 -> 64 -> 32 -> 1)
    model.add(Dense(128))
    model.add(Activation(activation))
    
    model.add(Dense(64))
    model.add(Activation(activation))
    
    model.add(Dense(32))
    model.add(Activation(activation))
    
    # 출력층
    model.add(Dense(1, activation='sigmoid'))
    
    # 컴파일 (Adam 기본값 사용 추정)
    model.compile(optimizer=Adam(), loss='binary_crossentropy', metrics=['accuracy'])
    return model
model = create_model()

In [14]:
# 4. 학습 
print("2만 장 데이터로 학습 시작")
model.fit(train_gen_20k, epochs=5)

2만 장 데이터로 학습 시작
Epoch 1/5
625/625 [==============================] - 144s 230ms/step - loss: 0.4312 - accuracy: 0.8116
Epoch 2/5
625/625 [==============================] - 29s 46ms/step - loss: 0.3261 - accuracy: 0.8652
Epoch 3/5
625/625 [==============================] - 29s 47ms/step - loss: 0.2827 - accuracy: 0.8833
Epoch 4/5
625/625 [==============================] - 29s 47ms/step - loss: 0.2547 - accuracy: 0.8945
Epoch 5/5
625/625 [==============================] - 29s 47ms/step - loss: 0.2260 - accuracy: 0.9043


In [12]:
# 5. 전체 검증 데이터로 평가
print("📊 전체 검증 데이터(41,025장)에 대한 평가 결과:")
y_true = np.load(VAL_LABEL)
y_pred_prob = model.predict(val_gen_full)
y_pred = (y_pred_prob > 0.5).astype(int).flatten()

print(classification_report(y_true, y_pred, digits=4))

📊 전체 검증 데이터(41,025장)에 대한 평가 결과:
1283/1283 [==============================] - 307s 240ms/step
              precision    recall  f1-score   support

           0     0.7616    0.4345    0.5533     20560
           1     0.6031    0.8633    0.7101     20465

    accuracy                         0.6484     41025
   macro avg     0.6823    0.6489    0.6317     41025
weighted avg     0.6825    0.6484    0.6315     41025

